In [1]:
pip install pandas numpy matplotlib seaborn scikit-learn


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
customers = pd.read_csv('c:/Users/DELL/Downloads/Customers.csv')
products = pd.read_csv('c:/Users/DELL/Downloads/Products.csv')
transactions = pd.read_csv('c:/Users/DELL/Downloads/Transactions.csv')


In [4]:
# Check missing values
print(customers.isnull().sum())
print(products.isnull().sum())
print(transactions.isnull().sum())

# Convert dates
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])


CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64
ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64
TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


In [5]:
data = pd.merge(transactions, customers, on='CustomerID')
data = pd.merge(data, products, on='ProductID')


In [7]:
print(data.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')


In [8]:
customer_summary = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price_x': 'mean'
}).reset_index()


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(customer_summary.iloc[:, 1:])
similarity_df = pd.DataFrame(similarity_matrix, index=customer_summary['CustomerID'], columns=customer_summary['CustomerID'])


In [10]:
lookalikes = {}
for customer in similarity_df.index[:20]:  # First 20 customers
    similar_customers = similarity_df.loc[customer].sort_values(ascending=False).index[1:4]
    scores = similarity_df.loc[customer, similar_customers]
    lookalikes[customer] = list(zip(similar_customers, scores))

lookalike_df = pd.DataFrame.from_dict(lookalikes, orient='index', columns=['Lookalike 1', 'Lookalike 2', 'Lookalike 3'])
lookalike_df.to_csv('FirstName_LastName_Lookalike.csv', index=True)
